# Developing an LLM application(RAG) using Langchain, (Google Gemini)

## Task 1: Import Libraries

In [3]:
import google.generativeai as genai
from IPython.display import display
from IPython.display import Markdown
import textwrap
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings, ChatGoogleGenerativeAI
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA 


In [7]:
from dotenv import load_dotenv
import os

load_dotenv()

api_key=os.getenv("Get_Api_Key")

genai.configure(api_key=api_key)
model = genai.GenerativeModel('gemini-pro')

## Task 2: Ask The Questions Using Prompts

In [2]:
response = model.generate_content("Explain Generative AI with 3 bullet points")
print(response.text)
Markdown(response.text)

I0000 00:00:1733399896.981744     195 config.cc:230] gRPC experiments enabled: call_status_override_on_cancellation, event_engine_dns, event_engine_listener, http2_stats_fix, monitoring_experiment, pick_first_new, trace_record_callops, work_serializer_clears_time_cache


InvalidArgument: 400 API key not valid. Please pass a valid API key. [reason: "API_KEY_INVALID"
domain: "googleapis.com"
metadata {
  key: "service"
  value: "generativelanguage.googleapis.com"
}
, locale: "en-US"
message: "API key not valid. Please pass a valid API key."
]

## Task 3: Chat With Gemini And Retrieve The Chat History

In [ ]:
hist = model.start_chat()
response = hist.send_message("Hi! Give me a recipe to make a margeritta pizza from scratch.")
Markdown(response.text)

for i in hist.history:
    print(i)
    print('\n\n')
i.parts[0].text 

model.count_tokens("Now please help me find the nearest supermarket from where I can buy the ingrediants.")

## Task 4: Experiment With The Temperature Parameter

In [ ]:
def get_response(prompt, generation_config={}):
    response = model.generate_content(contents=prompt, 
    generation_config=generation_config)
    return response

for temp in [0.0, 0.25, 0.5, 0.75, 1.0]:
  config = genai.types.GenerationConfig(temperature=temp)
  result = get_response("Explain the concepts of XGBoost and Random Forest with real-life use cases", generation_config=config)

  print(f"\n\nFor temperature value {temp}, the results are: \n\n")
  display(Markdown(result.text))

## Task 5: Experiment With Maximum Output Tokens

In [ ]:
def get_response(prompt, generation_config={}):
    response = model.generate_content(contents=prompt, generation_config=generation_config)
    return response
for m_o_tok in [1, 50, 100, 150, 200]:
    config = genai.types.GenerationConfig(max_output_tokens=m_o_tok)
    result = get_response("Explain the concepts of XGBoost and Random Forest with real-life use cases", generation_config=config)

    print(f"\n\nFor max output token value {temp}, the results are: \n\n")
    display(Markdown(result.text))

## Task 6: Experiment With the top_k Parameter

In [ ]:
def get_response(prompt, generation_config={}):
    response = model.generate_content(contents=prompt, 
    generation_config=generation_config)
    return response

for k in [1, 4, 16, 32, 40]:
    config = genai.types.GenerationConfig(top_k=k)
    result = get_response("Explain the concepts of XGBoost and Random Forest with real-life use cases", generation_config=config)

    print(f"\n\nFor top k value {temp}, the results are: \n\n")
    display(Markdown(result.text))

## Task 7: Experiment With the top_p Parameter

In [ ]:
def get_response(prompt, generation_config={}):
    response = model.generate_content(contents=prompt, 
    generation_config=generation_config)
    return response

for p in [0, 0.2, 0.4, 0.8, 1]:
    config = genai.types.GenerationConfig(top_p=p)
    result = get_response("Explain the concepts of XGBoost and Random Forest with real-life use cases", generation_config=config)

    print(f"\n\nFor top p value {temp}, the results are: \n\n")
    display(Markdown(result.text))

## Task 8: Experiment With the candidate_count Parameter

In [ ]:
config = genai.types.GenerationConfig(candidate_count=1)
result = get_response("Explain the concepts of XGBoost and Random Forest with real-life use cases", generation_config=config)
Markdown(result.text)

## Task 9: Introduction to Retrieval Augmented Generation

For the text-generation chatbot you are building, you’ll use retrieval-augmented generation (RAG), where a model extracts specific information to generate more relevant responses. The source of information can be a corpus, a document, a database, a website, or anything related.

There are some strong advantages of using RAG, like:

Enhanced contextual responses, increasing the LLM capabilities.

Retrieval of up-to-date information since it retrieves information from the website, document, or corpus.

Reduced hallucinations, which is a problem with the LLMs.

LangChain’s modular design helps integrate various document loaders, embedding models, vector stores, and LLM providers. In the coming tasks, you’ll use the tools provided by LangChain in the order specified below:

Load the PDF document (from where you will retrieve the specific data) using the PyPDFLoader.

Extract the texts using RecursiveCharacterTextSplitter to have meaningful chunks of text for further processing.

Use GoogleGenerativeAIEmbeddings to generate the embeddings for the extracted texts that will be used for similarity search in RAG.

Use Chroma (a vector database) to store the created embeddings so that they can be used to retrieve the relevant information later when needed.

Use the RetrievalQA function to build the question-answering system for retrieval (finding relevant context) with generation (generating an answer based on the retrieved context). This function will use ChatGoogleGenerativeAI with the Gemini Pro model to generate responses from the retrieved context.

## Task 10: Load the PDF and Extract the Texts

In [ ]:
CHUNK_SIZE = 700
CHUNK_OVERLAP = 100
pdf_path = "https://www.analytixlabs.co.in/assets/pdfs/Data_Engineering%20&_Other_Job_Roles-AnalytixLabs.pdf"

In [ ]:
pdf_loader = PyPDFLoader(pdf_path)
split_pdf_document = pdf_loader.load_and_split()

In [ ]:
# Splitting text into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=CHUNK_SIZE, chunk_overlap=CHUNK_OVERLAP)
context = "\n\n".join(str(p.page_content) for p in split_pdf_document)
texts = text_splitter.split_text(context)

## Task 11: Create the Gemini Model and Create the Embeddings

In [ ]:
gemini_model = ChatGoogleGenerativeAI(model='gemini-pro', google_api_key=<your_api_key>, temperature=0.8)

In [ ]:
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001", google_api_key=<your_api_key>)

In [ ]:
vector_index = Chroma.from_texts(texts, embeddings)
retriever = vector_index.as_retriever(search_kwargs={"k" : 5})

## Task 12: Create the RAG Chain and Ask Query

In [ ]:
qa_chain = RetrievalQA.from_chain_type(gemini_model, retriever=retriever, return_source_documents=True)

In [ ]:
# Example usage 
question = "Which tools do Data Engineers primarily work with?"
result = qa_chain.invoke({"query": question})
print("Answer:", result["result"])